In [1]:
import numpy as np
import os
import pandas as pd
import re
import pickle

In [2]:
episodes = []
for (dirpath, dirnames, filenames) in os.walk('C:\\Users\\Nirvan S P Theethira\\Desktop\\Mimic\\data\\transcripts_friends\\season_all\\'):
    for file in filenames:
        if '.csv' in file:
            episodes.append(dirpath+file)

In [5]:
def readFile(lines, person):
    scene = ''
    preChar = ''
    preScene = ''
    dialogue = ''
    # dataF = {'scene':[],'person':[], 'dialogue':[]}
    dataF = []
    dataRaw = ''
    prevDialogue = ''
    inputs = []
    outputs = []
    for line in lines:
        line = re.sub('\n','',line)
        if re.search(r"^\[[\w\W]+\]$", re.sub('\n','',line)):
            scene = line
        if re.search('^[a-zA-Z]+:',line) and len(scene)>0:
            charachter = re.findall('^[a-zA-Z]+:',line)[0]
            charachter = re.sub(':','',charachter).upper()
            if preChar!=charachter and len(preChar)!=0:
                if preChar==person:
                    dialogue = re.sub('\((.*?)\)','',dialogue)
                    dialogue = re.sub("[’']","",dialogue)
                    dialogue = re.sub('^\s+','',dialogue)
                    dialogue = re.sub('\s+$','',dialogue)
                    dataRaw += prevDialogue+'\t'+dialogue+'\n'
                    inputs.append(prevDialogue)
                    outputs.append(dialogue)
                dataF.append([preChar.upper(),dialogue])
                prevDialogue = re.sub('\((.*?)\)','',dialogue)
                prevDialogue = re.sub("[’']","",prevDialogue)
                prevDialogue = re.sub('^\s+','',prevDialogue)
                prevDialogue = re.sub('\s+$','',prevDialogue)
                dialogue=''

            dialogue += re.sub('^[a-zA-Z]+:','',line)
            preChar = charachter
    return dataRaw, inputs, outputs

In [14]:
inputs = []
outputs = []
person = 'ROSS'
for episode in episodes[0:-2]:
#     print(episode)
    with open(episode,encoding='utf8') as f:
        lines = f.readlines()
        _, inp, out = readFile(lines, person)
        inputs+=inp
        outputs+=out
print(inputs[0:3])
print(outputs[0:3])
print(len(inputs),len(outputs))

['Well, okay, Mikes taking a shower, which by the way theres no law against. And then were gonna grab some food, so if you want...', 'Ooh... Dr. Geller!', 'You gotta hear this, its great... Its like free porn!']
['... finally...', 'God, youre amazing... I didnt even have to ask you to call me that.', 'Weh...*sigh*']
8617 8617


In [15]:
trainTestSplit = 0.8
pickle.dump(inputs[:int(len(inputs)*trainTestSplit)], 
            open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\{}Data\\{}InputTrain.pkl'.format(person.lower(),person.lower()), 'wb'))
pickle.dump(inputs[int(len(inputs)*trainTestSplit):], 
            open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\{}Data\\{}InputTest.pkl'.format(person.lower(),person.lower()), 'wb'))
pickle.dump(outputs[:int(len(inputs)*trainTestSplit)], 
            open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\{}Data\\{}OutputTrain.pkl'.format(person.lower(),person.lower()), 'wb'))
pickle.dump(outputs[int(len(inputs)*trainTestSplit):], 
            open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\{}Data\\{}OutputTest.pkl'.format(person.lower(),person.lower()), 'wb'))

In [4]:
import re
class Preprocessor:
    def __init__(self, lower=False, keepPunct='[.,!?;]'):
        self.toLower = lower
        self.keepPunct = keepPunct
    
    def cleanTexts(self, textList, tokens=None):
        cleanText = []
        for sent in textList:
            sent = re.sub('[0-9]','',sent)
            if self.toLower:
                sent = sent.lower()
            words = re.findall(r"[\w]+|"+self.keepPunct, sent)
            if tokens:
                words = [tokens[0]]+words+[tokens[1]]
            cleanText.append(' '.join(words))
        return cleanText 

In [5]:
import pickle
import pandas as pd
person = 'joey'
genericQuestions = pickle.load(
    open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\{}Data\\genericQuestionsTrain.pkl'.format(person), 'rb'))
genericAnswers = pickle.load(
    open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\{}Data\\genericAnswersTrain.pkl'.format(person), 'rb'))
personInput = pickle.load(
    open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\{}Data\\{}InputTrain.pkl'.format(person,person), 'rb'))         
personOutput = pickle.load(
    open('C:\\Users\\Nirvan S P Theethira\\Desktop\\MiMic\\data\\{}Data\\{}OutputTrain.pkl'.format(person,person), 'rb'))

In [6]:
p = Preprocessor()
df = pd.DataFrame(data={'Generic Questions':p.cleanTexts(genericQuestions[10:20]),'Generic Answers':p.cleanTexts(genericAnswers[10:20], tokens=['<START>','<END>'])})
df

,Generic Questions,Generic Answers
0,What s up ?,<START> Not much . <END>
1,you are not put together,<START> i have always thought whoever did it c...
2,Tell me a joke,<START> what do you get when you cross a road ...
3,Who uses super computers ?,<START> Anybody who wants to work with large n...
4,how far is the sun,<START> the sun is about million miles from ea...
5,leo tolstoy,<START> Is one of Russia s greatest writers . ...
6,what is chemistry,<START> the science of mixing chemicals . <END>
7,Bend over,<START> That s personal ! <END>
8,do androids dream of electric sheep,<START> That is a good story by Philip K . Dic...
9,what is context,"<START> Context is hard . It s hard , and no o..."


In [9]:
df = pd.DataFrame(data={'Charachter Input':p.cleanTexts(personInput[0:5]),'Charachter Output':p.cleanTexts(personOutput[0:5], tokens=['<START>','<END>'])})
df

,Charachter Input,Charachter Output
0,Ooh . . . I love Barbados !,<START> Ooh . . . I cant believe Im kissing yo...
1,"Hey , you know , before you said that nothing ...","<START> Well , I only said that because of Ros..."
2,"Rach , you there ?","<START> Oh my God , its Ross . What are we gon..."
3,"Oh , ju ju just stay calm . Just be calm . For...",<START> No idea what it means . <END>
4,"Yeah , sure . . .",<START> I dont believe this . . . Have you guy...
